Welcome to the app, it serves as an easier alternative to see how to generate data and how the algorithm fits it.
It is not representative of the power of the app, but a nice starter.

In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
from scipy import integrate
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from lmfit import minimize, Parameters, Parameter, report_fit
from sklearn.metrics import r2_score
import random
import json
import time as pytime

In [2]:
# I declare some general variables the simulations use
P = 50
KH= 1282
CH2 = P/KH
R= 8.314
timespan = [0, 1, 3, 5, 7, 9, 11, 13, 15, 19, 23, 27, 31, 40, 50, 60, 90, 120, 150, 180, 240, 300, 360, 440, 530, 620, 810, 900, 955, 1246]
time = timespan[-1]

All the cells with functions are explained in the main algorithms

In [3]:
def preprocess_matrix(matrix,ks,reactions,concs,to_run):
    reactions = dict(reactions)
    matrix = dict(matrix)
    for key,dic in matrix.items():
        matrix[key] = dict(dic)
    ks = list(ks)
    concs = list(concs)
    
    for val in matrix.values():
        keylist = []
        for key in val.keys():            
            if key not in to_run.keys():
                keylist.append(key)                
        for pos in keylist:
            del val[pos]
            
    
    keylist = []
    for key in reactions.keys():            
        if key not in to_run.keys():
            keylist.append(key) 
    for pos in keylist:
        del reactions[pos]
        ks[pos-1] = 0

    
    keylist = []
    i=0
    poslist = []
    for key,dic in matrix.items():
        if dic == {}:
            keylist.append(key)
            poslist.append(i)
        i+=1
        
    for key in keylist:
        del matrix[key]
    poslist.reverse()
    for pos in poslist:
        del concs[pos]
        
    return matrix, ks, reactions, concs

In [4]:
def preprocess_reactions(reactions,to_run):
    for reaction in reactions.keys():
        reactions[reaction] = reactions[reaction][to_run[reaction]-1]
    return reactions

In [5]:
def diff(x, concs, ks,reacts,matrix):
    dt_list = []
    for val in (list(matrix.values())):
        tot = 0
        for key,value in val.items():
            react = reacts[key]
            tot += value*eval(react)
        dt_list.append(tot)
    for i in range(len(concs)-len(dt_list)):
        dt_list.append(0)
    return dt_list

In [6]:
class ODE_systems:
    def __init__(self, reactions, matrix,ks,concs,to_run = None):
        self.reactions = reactions
        self.matrix = matrix
        self.ks = ks
        self.concs = concs
        if to_run != None:
            self.to_run = to_run
        else:
            self.to_run = None
        
    def run_ODEs(self):
        to_ins = (list(self.matrix.values())[0])
        if self.to_run != None:
            for run in self.to_run:
                new_matrix, new_ks, new_reactions, new_concs = preprocess_matrix(self.matrix,self.ks,self.reactions,self.concs,run)
                new_reactions = preprocess_reactions(new_reactions,run)
                sol = integrate.solve_ivp(diff, [0,time], new_concs, t_eval = timespan, args=[new_ks,new_reactions,new_matrix])    
        else:
            sol = integrate.solve_ivp(diff, [0,time], self.concs, t_eval = timespan, args=[self.ks,self.reactions,self.matrix])
        return sol
    
    def get_matrices(self):
        systems_list = []
        if self.to_run != None:
            for run in self.to_run:
                new_matrix, new_ks, new_reactions, new_concs = preprocess_matrix(self.matrix,self.ks,self.reactions,self.concs,run)
                new_reactions = preprocess_reactions(new_reactions,run)
                systems_list.append([dict(new_matrix), list(new_ks), dict(new_reactions), list(new_concs), len(new_ks)])
        else:
            systems_list.append([self.matrix, self.ks, self.reactions, self.concs])
        return systems_list

In [7]:
def get_A(ktrue, E_act, ref_temp):
    R = 8.314
    T= ref_temp
    A = ktrue/(np.exp(-E_act/(ref_temp*R)))
    return A

In [8]:
def get_noise(t,var,stat):
    length = len(t)
    static_noise = np.random.normal(0,stat,length)
    variable_noise = np.random.normal(0,var,length)
    return static_noise, variable_noise

In [9]:
def make_data(system,timespan ,Ts,inits,ks,kin_params = None,var_noise = 0, stat_noise = 0,other_vars = None):
    time = timespan[-1]
    kin_params[1] = np.array(kin_params[1])
    
    if kin_params != None:
        pre_consts = get_A(ks,kin_params[1],kin_params[0])
    
    sol_list = []
    noisy_data_lists = []
    for init in inits:
        for T in Ts:
            noisy_data = []
            if kin_params != None:
                rate_consts = pre_consts*np.exp(-kin_params[1]/(R*T))
            else:
                rate_consts = ks
            sol = integrate.solve_ivp(system, [0,time], init, t_eval = timespan, args=[rate_consts])
            indexes = []
            
            for i in range(len(sol.y)):
                if sol.y[i][-1] == sol.y[i][0]:
                    indexes.append(i)
            indexes.reverse()
            sol.y = list(sol.y)
            if indexes != []:
                for index in indexes:
                    del sol.y[index]
                sol.y = np.array(sol.y)
            for line in sol.y:
                static_noise,variable_noise = get_noise(timespan,var_noise,stat_noise)
                noisy_data.append(list(line + static_noise + np.multiply(line,variable_noise)))
            data_dict = {"t" : list(timespan), "y": list(noisy_data), "T": T, "init" : init}
            noisy_data_lists.append(data_dict)
            

        
    noisy_data_lists = np.array(noisy_data_lists)
    return noisy_data_lists

In [10]:
def change_coord(data):
    rate_lists = []
    for line in range(len(data)):
        rate_list = []
        for i in range(len(data[line])-1):
            rate_list.append((data[line][i+1]-data[line][i])/(timespan[i+1]-timespan[i]))   
        rate_lists.append(rate_list)

    return rate_lists

In [11]:
def get_r2(real,simul,num_lines):
    tot_score = 0
    real_rate, simul_rate = change_coord(real), change_coord(simul)
    for i in range(num_lines):
        tot_score += r2_score(real[i],simul[i]) +r2_score(real_rate[i],simul_rate[i])
    return tot_score

In [12]:
def kin_params_guesses(constants, Ts):
    Eact_list = []
    A_list = []
    for i in range(len(constants[0])):
        log_list = []
        for constant_list in constants:
            log_list.append(np.log(constant_list[i]))
        reciprocal_list = 1/np.array(Ts)
        lin_model = np.polyfit(reciprocal_list,log_list,1)
        slope = lin_model[0]
        intercept = lin_model[1]
        Eact_list.append(slope*8.314)
        A_list.append(np.exp(intercept))
    return Eact_list, A_list

In [13]:
def fit_sim(system,data, iterations):
    matrix,reacts,num_ks,init = system[0],system[2],len(system[1]), system[3]
    if len(data) <= 1:
        init_list = [data[0]['init']]
        T = data[0]['T']
        values_list = [data[0]['y']]
        timespan_list = [data[0]['t']]
        time_list = [timespan[-1]]
        max_score = len(matrix)*2
    else:
        T = data[0]['T']
        init_list = []
        values_list = []
        timespan_list = []
        time_list = []
        max_score = 0
        for entry in data:
            init_list.append(entry['init'])
            values_list.append(entry['y'])
            timespan_list.append(entry['t'])
            time_list.append(entry['t'][-1])
            max_score += 2*len(matrix)
            

    def residual(paras):

        """
        compute the residual between actual data and fitted data
        """
        param_list = []
        for param in paras:
            param_list.append(paras[param].value)
            

        
        score = 0
        for i in range(len(data)):
            sol = integrate.solve_ivp(diff, [0,time_list[i]], init_list[i], t_eval = timespan_list[i], args=[param_list,reacts,matrix])
            score = get_r2(values_list[i], sol.y, len(values_list))

    

        return max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score

    
    #here

    params = Parameters()
    for i in range(num_ks):
        params.add(f'k{i}',value=0.01, min=1e-10, max=1)


    results = minimize(residual, params, method='leastsq', max_nfev = iterations)  # leastsq nelder
    
    result_list = []
    for param in results.params:
        result_list.append(results.params[param].value)

    
    final_res = []
    
    for i in range(len(results.params)):
        final_res.append(results.params[f'k{i}'].value)
    
    return final_res, T

In [14]:
def get_lines(diff, param_list, reacts, matrix,T, data):
    ks_list = []
    half = int(len(param_list)/2)
    for i in range(half):
        ks_list.append(param_list[i]*np.exp(-param_list[half+i]/(R*T)))

    init= data['init']
    T = data['T']
    timespan = data['t']
    time_list = timespan[-1]
    lines = integrate.solve_ivp(diff, [0,time], init, t_eval = timespan, args=[ks_list,reacts,matrix])

    return lines

In [15]:
#for full fit
def fit_sim_full(system,data, temps, Eacts, As,iterations):
    matrix,reacts,num_ks,init = system[0],system[2],len(system[1]), system[3]
    R = 8.314
    max_score = 0
    for dataset in data:
        for run in dataset:
            max_score += 2*len(matrix)
    pre_consts = []
    E_acts = []
    
    As_names = []
    Eacts_names = []
    for i in range(num_ks):
        As_names.append(f'A{i+1}')
        Eacts_names.append(f'E{i+1}')

    
    def residual(paras):

        """
        compute the residual between actual data and fitted data
        """
        param_list = []
        
        for param in paras:
            param_list.append(paras[param].value)
        
        
        half = int(len(param_list)/2)
        for i in range(half):
            val = param_list[i]*np.exp(-param_list[half+i]/(R*temps[-1]))
            if val >1:
                return max_score,max_score,max_score,max_score,max_score,max_score,max_score,max_score,max_score,max_score,max_score,max_score,
        lines_list = []
        for i in range(len(data)):
            T_current = data[i]['T']
            lines = get_lines(diff,param_list,reacts,matrix,T_current,data[i])
            lines_list.append(lines)


        score = 0
        num = 0
        for run in data:
            score += get_r2(run['y'], lines_list[num].y,len(run['y']))
            num += 1
        return max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score,max_score-score

    params = Parameters()
    for i in range(len(As)):
        params.add(As_names[i],value=As[i], min=1e-5, max=1e5)
    for i in range(len(Eacts)):
        params.add(Eacts_names[i],value=-Eacts[i], min=1000, max=100000)    


    results = minimize(residual, params, method='leastsq', max_nfev = iterations)  # leastsq nelder
    
    result_list = []
    for param in results.params:
        result_list.append(results.params[param].value)

        
    rows = 2
    cols = 2
    row = 1
    col = 1
    fig = make_subplots(rows=2, cols=2)
    for i in range(len(data)):
        if len(data[0]['y']) > len(matrix.items()):
            T_current = data[i]['T']
            mock_run = get_lines(diff,result_list,reacts,matrix,T_current,data[i])
            for j in range(len(matrix.items())):
                fig.add_trace(go.Scatter(x=data[i]['t'],y=data[i]['y'][j], mode= 'markers', name= list(matrix.keys())[j][1:-2], legendgroup = '1'), row = row, col=col)
                fig.add_trace(go.Scatter(x=mock_run.t,y=mock_run.y[j], mode= 'lines', name =list(matrix.keys())[j][1:-2]), row = row, col=col)
            col +=1
            if col > cols:
                row += 1
                col =1
        else:
            T_current = data[i]['T']
            mock_run = get_lines(diff,result_list,reacts,matrix,T_current,data[i])
            for j in range(len(data[0]['y'])):
                fig.add_trace(go.Scatter(x=data[i]['t'],y=data[i]['y'][j], mode= 'markers', name =list(matrix.keys())[j][1:-2], legendgroup = '1'), row = row, col=col)
                fig.add_trace(go.Scatter(x=mock_run.t,y=mock_run.y[j], mode= 'lines', name =list(matrix.keys())[j][1:-2]), row = row, col=col)
            col +=1
            if col > cols:
                row += 1
                col =1
            
    sim = residual(results.params)[0]
    print(max_score-sim)
    
    report_fit(results)
    
    return results.params, fig

In [16]:
class fit_data():
    def __init__(self, data, systems):
        self.data = data
        self.systems = systems
        
    def fit_one(self,entry_num,system_num, iterations = 100):
        data = [self.data[entry_num]]
        system = self.systems[system_num]
        fit_sim(system,data,iterations)
        
    def fit_many_entries(self,entries,system_num, iterations = 100):
        for entry in entries:
            data = [self.data[entry]]
            system = self.systems[system_num]
            fit_sim(system,data,iterations)
    
    def fit_many_systems(self,entry_num,systems, iterations = 100):
        for system in systems:
            data = [self.data[entry_num]]
            system = self.systems[system]
            fit_sim(system,data,iterations)
    
    def fit_many(self,entries,systems, iterations = 100):
        for entry in entries:
            for system in systems:            
                data = [self.data[entry]]
                system = self.systems[system]
                fit_sim(system,data,iterations)
                
    def fit_full(self,system_num, iterations = 100, iterations_full = 20):
        system = self.systems[system_num]
        unique_temps = []
        data_list = []
        params_list = []
        T_list = []
        for entry in self.data:
            unique_temps.append(entry['T'])
        unique_temps = set(unique_temps)
        unique_temps = list(unique_temps)
        unique_temps.sort()
        for temp in unique_temps:
            hits = []
            for entry in self.data:
                if entry['T'] == temp:
                    hits.append(entry)
            data_list.append(hits)
        for pair in data_list:
            params, T = fit_sim(system,pair,iterations)
            params_list.append(params)
            T_list.append(T)
        Eacts, As = kin_params_guesses(params_list,T_list)
        print(Eacts,As)
        result, fig = fit_sim_full(system,self.data,T_list,Eacts,As, iterations_full)
        return result, fig
        
            

From here there are a lot of differential systems, these serve to be used to make data for the test.
In some programs there are functions to make data, in some there is imported data used.

In [17]:
def diff_1(x, init, rate_const):
    CE, CA1, CA2, CtE,Cprecat, Ccat, Ccatdead,CH2 = init
    k1= rate_const[0]
    dEdt = -k1*CE
    dA1dt = k1*CE
    dA2dt = k1*CE
    return dEdt, dA1dt, dA2dt,0,0,0,0,0

In [18]:
def diff_2(x, init, rate_const):
    CE, CA1, CA2,Ccatdead, Cprecat, CtE, Ccat,CH2 = init
    k1, k2 = rate_const[0], rate_const[1]
    dEdt = -k1*CE*Cprecat
    dA1dt = k1*CE*Cprecat
    dA2dt = k1*CE*Cprecat
    dprecatdt = -k2*Cprecat
    dcatdeaddt = k2*Cprecat
    return dEdt, dA1dt, dA2dt, dcatdeaddt, dprecatdt, 0,0,0

In [19]:
def custom(x, init, rate_const):
    CA,CB,CC,CD,CE,CF,CG,CH = init
    k1, k2= rate_const[0],rate_const[1]
    dAdt = -k1*CA
    dBdt = k1*CA-k2*CB*CC
    dCdt = k1*CA-k2*CB*CC
    dDdt = k2*CB*CC
    return dAdt, dBdt, dCdt, dDdt, 0, 0, 0, 0

And this is the app to make all the other inputs easy to see and interact with. The parameters in the app are used to change the base data,
after which the algorithm will fit the data with a specified system (comment where to specify which system in cell below). 
You can also load in different systems to fit the data with (also dentoted in the cell below).

Note that the app has limited amounts of constants and iterations to make it fast for exhibition purposes. The performance of the main program is leagues better.

Estimates will likely be off by large margin, but if similar data to the intial data is used it is a showcase of what the algorithm can do. Changing the constants and activation energies is most likely to give an erroneous result.

In [21]:
# app layout -----------------------------------
df = px.data.tips()
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Own plot demo"),
    
    html.Label([
        "temperatures:  ",
        html.Br(),
        html.Label('temperature 1:  '),
        dcc.Input(
            id='temperature_1', type = 'number',
            value=300, min = 1, max = 500
            ),
        html.Label('temperature 2:  '),
        dcc.Input(
            id='temperature_2', type = 'number',
            value=400, min = 1, max = 500
            )
    ]),
    html.Br(),
    html.Div([
        html.Label([
            "concentrations:  ",
            html.Br(),
            html.Label('substrate concentration 1:  '),
            dcc.Input(
                id='sub_conc_1', type = 'number',
                value=1, min = 0.01, max = 10
                ),
            html.Label('substrate concentration 2:  '),
            dcc.Input(
                id='sub_conc_2', type = 'number',
                value=1, min = 0.01, max = 10
                ),
            html.Br(),
            html.Label('catalyst concentration 1:  '),
            dcc.Input(
                id='cat_conc_1', type = 'number',
                value=1, min = 0.01, max = 10
                ),
            html.Label('catalyst concentration 2:  '),
            dcc.Input(
                id='cat_conc_2', type = 'number',
                value=1, min = 0.01, max = 10
                )
        ]),
        
    ]),
    html.Br(),
    html.Div([
        html.Label([
            "noise:  ",
            html.Br(),
            html.Label('Variable noise:  '),
            dcc.Input(
                id='var_noise', type = 'number',
                value=0.05, min = 0.0001, max = 1
                ),
            html.Label('Static noise:  '),
            dcc.Input(
                id='stat_noise', type = 'number',
                value=0.01, min = 0.0001, max = 1
                ),
        ]),
        
    ]),
    html.Br(),
    html.Div([
        html.Label([
            "kinetic parameters:  ",
            html.Br(),
            html.Label('kinetic constant 1:  '),
            dcc.Input(
                id='k_1', type = 'number',
                value=0.01, min = 0.0001, max = 1
                ),
            html.Label('activation energy 1:  '),
            dcc.Input(
                id='E_1', type = 'number',
                value=10000, min = 1000, max = 100000
                ),
            html.Br(),
            html.Label('kinetic constant 2:  '),
            dcc.Input(
                id='k_2', type = 'number',
                value=0.01, min = 0.0001, max = 1
                ),
            html.Label('activation energy 2:  '),
            dcc.Input(
                id='E_2', type = 'number',
                value=10000, min = 1000, max = 100000
                ),
        ]),
        
    ]),
    
    html.Br(),
    html.Div([
        html.Label(['which system to run',
        dcc.Input(
                id='system', type = 'number',
                value=0
            ),
        ]),
    ]),
        
    
    html.Div([
        html.Label([
            'outputs:',
        dcc.Store(id='base_data'),
        dcc.Graph(id='graph')
        ])
    ]),
    
    html.Div([
        html.Label([
            'output fits:',
        dcc.Graph(id='fit graph')
        ])
    ])
        
])
#--------------------------------------------
# App back-end logic --------------------------------------------------
@app.callback(
    Output('base_data', 'data'),
    Input("temperature_1", "value"),
    Input('temperature_2', 'value'),
    Input('sub_conc_1', 'value'),
    Input('sub_conc_2', 'value'),
    Input('cat_conc_1', 'value'),
    Input('cat_conc_2', 'value'),
    Input('var_noise', 'value'),
    Input('stat_noise', 'value'),
    Input('k_1', 'value'),
    Input('E_1', 'value'),
    Input('k_2', 'value'),
    Input('E_2', 'value'),
)
def update_figure(temp_1,temp_2,sub_1,sub_2,cat_1,cat_2,var_noise,stat_noise,k1,E1,k2,E2):
    pytime.sleep(1)
    differential_system = custom #fill in the name for the differential system you want to generate data
    data = make_data(differential_system,timespan ,[temp_1,temp_2],[[sub_1,0,0,0,cat_1,0,0,0],[sub_2,0,0,0,cat_2,0,0,0]],[k1,k2],
                     var_noise = var_noise, stat_noise = stat_noise,kin_params = [temp_1,[E1,E2]])
    return json.dumps(data.tolist())

@app.callback(
    Output('graph', 'figure'),
    Input("base_data", "data")
)
def update_figure(data):
    data = json.loads(data)
    fig = make_subplots(rows=2,cols=2)
    for i in range(len(data[0]['y'])) : fig.add_trace(go.Scatter(x= data[0]['t'], y = data[0]['y'][i]), row= 1, col=1)
    for i in range(len(data[0]['y'])) : fig.add_trace(go.Scatter(x= data[1]['t'], y = data[1]['y'][i]), row= 1, col=2)
    for i in range(len(data[0]['y'])) : fig.add_trace(go.Scatter(x= data[2]['t'], y = data[2]['y'][i]), row= 2, col=1)
    for i in range(len(data[0]['y'])) : fig.add_trace(go.Scatter(x= data[3]['t'], y = data[3]['y'][i]), row= 2, col=2)
    return fig

@app.callback(
    Output('fit graph', 'figure'),
    Input("base_data", "data"),
    Input('system', 'value')
)
def update_result(data, system_num):
    data = json.loads(data)

    #Three different systems the data can be fit with, wrong systems will give worse fits
    #notation of the systems is explained in the main document
    
#     #system 1
#     ks = [0.01209668296564242]
#     concs = [1,0,0]
#     reactions = {1:["ks[0]*concs[0]"]}
#     matrix = {"dEdt": {1:-1},"dA1dt": {1:1},"dA2dt": {1:1}}
#     to_run = [{1:1}]
    
#     #system 2
#     ks = [0.01209668296564242, 0.012690318715024702]
#     concs = [1,0,0,1,0]
#     to_run = [{1:1,2:1},{1:1}]
#     reactions = {1:["ks[0]*concs[0]"],
#                    2:["ks[1]*concs[3]"]}
#     matrix = {"dEdt": {1:-1},
#           "dA1dt": {1:1},
#           "dA2dt": {1:1},
#          "dprecatdt": {2:-1},
#          "dcatdt": {2:1}}
    


    # system 3, feel free to change things if you know how to
    ks = [0.01,0.01]
    concs = [0.1,0,0,0]
    reactions = {1 : ["ks[0]*concs[0]"],
                2 : ["ks[1]*concs[1]*concs[2]", 'ks[1]*concs[1]*concs[2]*concs[0]']}
    
    matrix = {"dAdt": {1:-1},
             "dBdt": {1:1, 2:-1},
             "dCdt": {1:1, 2:-1},
             "dDdt": {2:1}}
    #you can even run different versions of 1 differential system, change ' which system to run' in the app
    #to run the 2 different systems in to_run
    to_run = [{1:1, 2:1}, {1:1,2:2}]
    
    
    test = ODE_systems(reactions,matrix,ks,concs,to_run)
    test_fit_1 = fit_data(data, test.get_matrices())
    results,fig = test_fit_1.fit_full(system_num)
    
    

    return fig
#-------------------------------
# run the app in the notebook
app.run_server('inline')

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-9-d2e2de99a88c> in make_data(
    system=<function custom>,
    timespan=[0, 1, 3, 5, 7, 9, 11, 13, 15, 19, 23, 27, 31, 40, 50, 60, 90, 120, 150, 180, ...],
    Ts=[300, 400],
    inits=[[1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0]],
    ks=[None, 0.01],
    kin_params=[300, array([10000, 10000])],
    var_noise=0.05,
    stat_noise=0.01,
    other_vars=None
)
      4 
      5     if kin_params != None:
----> 6         pre_consts = get_A(ks,kin_params[1],kin_params[0])
        pre_consts = undefined
        global get_A = <function get_A at 0x000001E11C067790>
        ks = [None, 0.01]
        kin_params = [300, array([10000, 10000])]
      7 
      8     sol_list = []

<ipython-input-7-27856917e149> in get_A(
    ktrue=[None, 0.01],
    E_act=array([10000, 10000]),
    ref_temp=300
)
      2     R = 8.314
      3  